In [ ]:
# 随机森林
# 整个环境有利于画图
%matplotlib inline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
wine = load_wine()
wine.data.shape
wine.target
# 实例化
# 训练集导入实例化后的模型进行训练，使用的接口是fit
# 使用其他接口将测试集导入
x_train, x_valid, y_train, y_valid = train_test_split(wine.data, wine.target, test_size=0.3)
clf = DecisionTreeClassifier(random_state=0)
rfc = RandomForestClassifier(random_state=0)
clf.fit(x_train, y_train)
rfc.fit(x_train, y_train)
score_c = clf.score(x_valid, y_valid)
score_r = rfc.score(x_valid, y_valid)
print("Single Tree:{}".format(score_c))
print("Random Forest:{}".format(score_r))

In [ ]:

# 交叉验证
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
rfc = RandomForestClassifier(n_estimators=25)
rfc_s = cross_val_score(rfc, wine.data, wine.target, cv=10)

clf = DecisionTreeClassifier()
clf_s = cross_val_score(clf, wine.data, wine.target, cv=10)

plt.plot(range(1, 11), rfc_s, label="RandomForest")
plt.plot(range(1, 11), clf_s, label="DecisionTreeClassifier")
plt.legend()
plt.show()
##########################


In [ ]:
#解释为什么随机森林牛批
import numpy as np
from scipy.special import comb
# comb 是组合
np.array([comb(25, i) * (0.2 ** i) * (1 - 0.2) ** (25 - i) for i in range(13, 26)]).sum()
rfc = RandomForestClassifier(n_estimators=25, random_state=2)
rfc = rfc.fit(x_train, y_train)

# extimators_是看树的本身的状况，[0]是取第一颗树
rfc.estimators_[0].random_state
#rfc.estimators[]是tree的类型，放到dataframe里面会使模型消失
for i in range(len(rfc.estimators_)):
    print(rfc.estimators_[i].random_state) 

# oob方法测试,无需划分训练集和测试集
rfc = RandomForestClassifier(n_estimators=25, oob_score=True)
rfc = rfc.fit(wine.data, wine.target)
# 重要属性oob_score_
rfc.oob_score_


In [ ]:
# bagging相关
import numpy as np
x = np.linspace(0, 1, 20)
y = []

for epsilon in np.linspace(0, 1, 20):
    E = np.array([comb(25, i) * (epsilon ** i) * ((1 - epsilon) ** (25 - i)) for i in range(13, 26)]).sum()
    y.append(E)
plt.plot(x, y, 'o-', label='when estimators are different')
plt.plot(x, x, '--', label='if all estimators are same')
plt.xlabel("individual estimator's error")
plt.ylabel("RandomForest's error")
plt.legend()
plt.show()



In [ ]:
#随机森林随机树
from sklearn.datasets import load_boston # 一个标签是连续性变量的数据集，字典
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
import sklearn
boston = load_boston()

regressor = RandomForestRegressor(n_estimators=100, random_state=0) #实例化
cross_val_score(regressor, boston.data, boston.target, cv=10, scoring='neg_mean_squared_error') #交叉验证
#import sklearn
sorted(sklearn.metrics.SCORERS.keys())# sklearn.metrics sklearn中的模型评估（打分）指标列表,忘了指标可以用这个查找


In [ ]:
# 填补随机值
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

dataset = load_boston()
dataset.data.shape
# 总共506*13
x_full, y_full = dataset.data, dataset.target
n_samples = x_full.shape[0]
n_features = x_full.shape[1]
# 首先我们需要确定我们希望放入的缺失数据的比例，在这里我们假设是50%，那总共要有3289个缺失值
rng = np.random.RandomState(0)# 确认一种随机模式，数字随便搞
missing_rate = 0.5
#np.floor 向下取整，但是返回浮点数
n_missing_samples = int(np.floor(n_samples * n_features * missing_rate))
# 所有数据要随机遍布在数据集的各行各列中，每一个缺失值会需要一个行索引和一个列索引
# 如果能够创造一个数组，包含3289个分布再0-506中间的行索引，和3289个分布在0-13之间的列索引
# 那我们就可以利用索引来为数据中的任意3289个位置赋空值
# 然后我们用0，均值和随机森林来填写这些缺失值，然后查看回归的结果如何

missing_features = rng.randint(0, n_features, n_missing_samples)
missing_samples = rng.randint(0, n_samples, n_missing_samples)

#missing_samples = rng.choice(dataset.data.shape[0], n_missing_samples, replace=False)
# 采样了3289个数据，远远超过我们的样本量，所以我们使用随机抽取，若是我们需要的数据量小于我们的样本量，
#那可以。random.choice不重复抽样，使得数据更加分散，不会集中在某些行中
# replace=false 请不要重复

# 不改变本身
x_missing = x_full.copy()
y_missing = y_full.copy()

x_missing[missing_samples, missing_features] = np.nan
# 转换成dataframe为了后续便于操作，numpy矩阵运算超快，但是索引确实不好用
x_missing = pd.DataFrame(x_missing)
# 使用均值和0进行填补
from sklearn.impute import SimpleImputer
# 空值怎么表示，策略是mean，还可以填其他的
imp_mean = SimpleImputer(missing_values=np.nan, strategy="mean")
# 训练fit+导出predict 》》》特殊接口fit_transform
x_missing_mean = imp_mean.fit_transform(x_missing)

#x_missing_mean.isnull() #错误因为是ndarry
pd.DataFrame(x_missing_mean).isnull().sum()
#布尔值 false = 0， true = 1

# 使用0进行填补,constant表示填充策略是常数
imp_0 = SimpleImputer(missing_values=np.nan, strategy="constant", fill_value=0)
x_missing_0 = imp_0.fit_transform(x_missing)



In [ ]:
"""
使用随机森林填充缺失值
任何回归都是从矩阵中学习，然后求解连续型标签y的过程，因为算法认为矩阵与标签之间存在某种联系。
实际上标签和特征是可以相互转化的，比如地区预测房价，也可以房价预测地区。回归填补缺失值正是利用这个思想
对于一个有n维他特征的数据，特征r有缺失值，把r当作标签，其他n-1个特征和原本的标签组成新的特征矩阵，r没缺失的部分就是y_train,可以作为标签，缺失的就是我们要预测的部分
特征r不缺失的值对应其他n-1个他特征和本来的标签作为x-train，特征r不确实的值y-train
特征r缺失的值对应其他n-1个他特征和本来的标签作为x-valid，特征r确实的值y-valid
这种情况对某一个特征大量缺失其他特征完整非常适用
如果除了特征r，其他特征很完整的话，？
答案是遍历所有特征，从缺失值最少的开始填补（需要准确性特征最少）
填补一个特征时，先将其他特征缺失值用0代替，没完成一次回归，就将预测值放到原本的特征阵中，在继续填补下一个特征。不断完整矩阵
"""
x_missing_reg = x_missing.copy()
#  找出数据中，缺失值从小到大排列的特征的顺序
sortindex = np.argsort(x_missing_reg.isnull().sum(axis=0)).values
# argsort 会返回从小到大排序的顺序所对应的索引，。values就是把索引给取出来
# sort就直接给每个缺失值是多少的排序，但没有索引 axis=0 就是对行相加操作，axis=1就是对列相加操作

####开始填充

# 下面的填充从小到多，这里可以自己还原
for i in sortindex:

# 构建我们的新特征矩阵和新标签（没有被选中的填充的特征+原始的标签）和新标签（被选中去填充的标签）
# 复制一个表是用来填0的
df = x_missing_reg

fillc = df.iloc[:, 6]
# 新特征矩阵 df.columns返回列索引, 连链接起来，dataframe（y_full）也变成这个类型. 左右拼起来
df = pd.concat([df.iloc[:, df.columns != 6], pd.DataFrame(y_full)], axis=1)
# 在新特征矩阵中，对含有缺失值的列，进行0的填补，实例化，训练加导出
df_0 = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0).fit_transform(df) 
# 找出我们的训练集和测试集
# 是被选中的要填充的特征中（现在是我们的标签），存在的那些值，非空值
y_train = fillc[fillc.notnull()]
# fillc.isnull()返回的是true和false
y_valid = fillc[fillc.isnull()]
# 在新特征矩阵上，被选出来的要填充的特征的非空值所对应的记录
x_train = df_0[y_train.index, :]
x_valid = df_0[y_valid.index, :]

# 用随机森林回归来填补缺失值
rfc = RandomForestRegressor(n_estimators=100)
rfc = rfc.fit(x_train, y_train)
y_predict = rfc.predict(x_valid) # 就是我们要填写空值的那些值
# 将补好的特征返回到我们的原始特征矩阵中
# pd.DataFrame(df_0)
x_missing_reg.loc[x_missing_reg.iloc[:, 6].isnull(), 6] = y_predict

# x_missing_reg.isnull().sum()


In [ ]:
################# 合成的最终程序
x_missing_reg = x_missing.copy()
#  找出数据中，缺失值从小到大排列的特征的顺序
sortindex = np.argsort(x_missing_reg.isnull().sum(axis=0)).values
# argsort 会返回从小到大排序的顺序所对应的索引，。values就是把索引给取出来
# sort就直接给每个缺失值是多少的排序，但没有索引 axis=0 就是对行相加操作，axis=1就是对列相加操作

####开始填充

# 下面的填充从小到多，这里可以自己还原
for i in sortindex:

    # 构建我们的新特征矩阵和新标签（没有被选中的填充的特征+原始的标签）和新标签（被选中去填充的标签）
    # 复制一个表是用来填0的
    df = x_missing_reg

    fillc = df.iloc[:, i]
    # 新特征矩阵 df.columns返回列索引, 连链接起来，dataframe（y_full）也变成这个类型. 左右拼起来
    df = pd.concat([df.iloc[:, df.columns != i], pd.DataFrame(y_full)], axis=1)
    # 在新特征矩阵中，对含有缺失值的列，进行0的填补，实例化，训练加导出
    df_0 = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0).fit_transform(df) 
    # 找出我们的训练集和测试集
    # 是被选中的要填充的特征中（现在是我们的标签），存在的那些值，非空值
    y_train = fillc[fillc.notnull()]
    # fillc.isnull()返回的是true和false
    y_valid = fillc[fillc.isnull()]
    # 在新特征矩阵上，被选出来的要填充的特征的非空值所对应的记录
    x_train = df_0[y_train.index, :]
    x_valid = df_0[y_valid.index, :]

    # 用随机森林回归来填补缺失值
    rfc = RandomForestRegressor(n_estimators=100)
    rfc = rfc.fit(x_train, y_train)
    y_predict = rfc.predict(x_valid) # 就是我们要填写空值的那些值
    # 将补好的特征返回到我们的原始特征矩阵中
    # pd.DataFrame(df_0)
    x_missing_reg.loc[x_missing_reg.iloc[:, i].isnull(), i] = y_predict

    # x_missing_reg.isnull().sum()

In [ ]:
##########对比几种模型
X = [x_full, x_missing_mean, x_missing_0, x_missing_reg]
mse = []
for x in X:
    estimator = RandomForestRegressor(random_state=0, n_estimators=100)#实例化
    scores = cross_val_score(estimator, x, y_full, scoring="neg_mean_squared_error",cv=5).mean()
    mse.append(scores * -1)
[*zip(['x_full', 'x_missing_mean', 'x_missing_0', 'x_missing_reg'], mse)]

In [ ]:
x_labels = [
    'Full data',
    'Mean Imputation',
    'Zero Imputation',
    'Regressor Imputation'
]
colors = ['r', 'g', 'b', 'orange']

plt.figure(figsize=(12,6))
ax = plt.subplot(111)
for i in np.arange(len(mse)):
    # barh是从左向右找到条形图,align 是条形图的粗
    ax.barh(i, mse[i], color=colors[i], alpha=0.6, align="center")
ax.set_title('Imputation Techniques with Boston Data')
# 因为是横过来的，看左右
ax.set_xlim(left=np.min(mse) * 0.9, right=np.max(mse) * 1.1)
# 看刻度，有四个刻度
ax.set_yticks(np.arange(len(mse)))
# 
ax.set_xlabel('mse')
# 这句话没什么用
ax.invert_yaxis()

ax.set_yticklabels(x_labels)
plt.show()

